In [1]:
import requests
import pandas as pd
import json
from copy import copy

In [2]:
addr = 'http://127.0.0.1:8000/'
endpoint = f"{addr}/geocoding/api/v1/address/"

In [3]:
headers = {'Content-Type': 'text/csv'}
response = requests.post(endpoint,
                         data=open('city.csv' ,'rb'),
                         headers=headers)
response.status_code

206

In [4]:
response_data = json.loads(response.content)

In [5]:
len(response_data['errors'])

104

In [6]:
response_data['errors'][0]

{'INVALID_DATA': {'foundation_year': ['Введите правильное число.']},
 'address': {'address': 'Архангельская обл, г Мезень',
  'postal_code': '164750',
  'country': 'Россия',
  'federal_district': 'Северо-Западный',
  'region_type': 'обл',
  'region': 'Архангельская',
  'area_type': 'р-н',
  'area': 'Мезенский',
  'city_type': 'город',
  'city': 'Мезень',
  'settlement_type': '',
  'settlement': '',
  'kladr_id': '2901200100000',
  'fias_id': '8a49f67f-6c48-4160-bd1d-ebc188a69269',
  'fias_level': '4',
  'capital_marker': '1',
  'okato': '11242501000',
  'oktmo': '11642101001',
  'tax_office': '2932',
  'timezone': 'UTC+3',
  'geo_lat': '65.8398078',
  'geo_lon': '44.2532273',
  'population': '3599',
  'foundation_year': 'XVI век'}}

In [7]:
# Все ли ошибки с валидацией?
sum('INVALID_DATA' in item for item in response_data['errors'])

104

In [8]:
data = []
for item in response_data['errors']:
    address = copy(item['address'])
    error_fields = {f"_{field}":True for field in item['INVALID_DATA'].keys()}
    address.update(error_fields)
    data.append(address)
data = pd.DataFrame(data)
data.columns

Index(['address', 'postal_code', 'country', 'federal_district', 'region_type',
       'region', 'area_type', 'area', 'city_type', 'city', 'settlement_type',
       'settlement', 'kladr_id', 'fias_id', 'fias_level', 'capital_marker',
       'okato', 'oktmo', 'tax_office', 'timezone', 'geo_lat', 'geo_lon',
       'population', 'foundation_year', '_foundation_year', '_population'],
      dtype='object')

Ошибки только в полях: foundation_year и population

### population

In [9]:
data[data._population == True]

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year,_foundation_year,_population
18,"Респ Ингушетия, г Сунжа",386200,Россия,Северо-Кавказский,Респ,Ингушетия,,,город,Сунжа,...,26230835001,26720000001,603,UTC+3,43.3204196,45.0476331,64493[3],1845,NaN,True
44,"Ленинградская обл, Всеволожский р-н, г Кудрово",188691,Россия,Северо-Западный,обл,Ленинградская,р-н,Всеволожский,город,Кудрово,...,41212812002,41612155005,4703,UTC+3,59.9075226,30.5121008,13501[4],1926,NaN,True
45,"Ленинградская обл, Всеволожский р-н, г Мурино",188662,Россия,Северо-Западный,обл,Ленинградская,р-н,Всеволожский,город,Мурино,...,41212000101,41612428101,4703,UTC+3,60.044862,30.4571456,19775[4],1676,NaN,True
86,"Респ Татарстан, Верхнеуслонский р-н, г Иннополис",420500,Россия,Приволжский,Респ,Татарстан,р-н,Верхнеуслонский,город,Иннополис,...,92220509000,92620109001,1683,UTC+3,55.7521699,48.7446846,96[3],2012,NaN,True
87,"Респ Татарстан, г Кукмор",422110,Россия,Приволжский,Респ,Татарстан,р-н,Кукморский,город,Кукмор,...,92233501000,92633101001,1675,UTC+3,56.1861392,50.8970238,17700[4],1741,NaN,True


In [10]:
data.population.apply(lambda s: s.split('[')[0] if '[' in s else s)[data._population == True]

18    64493
44    13501
45    19775
86       96
87    17700
Name: population, dtype: object

In [11]:
data.population = data.population.apply(lambda s: s.split('[')[0] if '[' in s else s)
data.drop(columns='_population', inplace=True)

### foundation_year

In [12]:
data[data._foundation_year == True]

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,capital_marker,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year,_foundation_year
0,"Архангельская обл, г Мезень",164750,Россия,Северо-Западный,обл,Архангельская,р-н,Мезенский,город,Мезень,...,1,11242501000,11642101001,2932,UTC+3,65.8398078,44.2532273,3599,XVI век,True
1,"Респ Башкортостан, г Давлеканово",453400,Россия,Приволжский,Респ,Башкортостан,р-н,Давлекановский,город,Давлеканово,...,1,80419000000,80622101,273,UTC+5,54.2226707,55.0312373,24040,1730-е,True
2,"Респ Башкортостан, г Учалы",453700,Россия,Приволжский,Респ,Башкортостан,р-н,Учалинский,город,Учалы,...,1,80455000000,80653101,256,UTC+5,54.3067375,59.4125461,37771,сер. XVIII века,True
3,"Респ Башкортостан, г Янаул",452800,Россия,Приволжский,Респ,Башкортостан,р-н,Янаульский,город,Янаул,...,1,80460000000,80659101,264,UTC+5,56.2650146,54.929907,26988,1730-е,True
4,"Брянская обл, г Сельцо",241550,Россия,Центральный,обл,Брянская,,,город,Сельцо,...,0,15425000000,15725000001,3245,UTC+3,53.3739884,34.1059172,17933,1870-е,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,"Респ Чеченская, г Урус-Мартан",366500,Россия,Северо-Кавказский,Респ,Чеченская,р-н,Урус-Мартановский,,,...,1,96234501000,96634101001,2033,UTC+3,43.1201923,45.5391777,49071,XVIII век,True
100,"Респ Чеченская, г Шали",366300,Россия,Северо-Кавказский,Респ,Чеченская,р-н,Шалинский,город,Шали,...,1,96237501000,96637101001,2000,UTC+3,43.1488691,45.9009629,47715,1378 или 1384,True
101,"Ярославская обл, г Данилов",152070,Россия,Центральный,обл,Ярославская,р-н,Даниловский,город,Данилов,...,1,78215501000,78615101001,7611,UTC+3,58.1860098,40.1795067,15861,15 век,True
102,"Ярославская обл, г Мышкин",152830,Россия,Центральный,обл,Ярославская,р-н,Мышкинский,город,Мышкин,...,1,78221501000,78621101,7612,UTC+3,57.7881684,38.4544224,5932,15 век,True


In [13]:
import re
roman_nums = r'([IVX]+)'

rom = {'I':1, 'V':5, 'X':10}

def roman_to_digits(roman:str) -> int:
    """Only for combinations from 'IVX'."""
    result = 0
    for ind,symbol in enumerate(roman):
        if ind + 1 == len(roman):
            return result + rom[symbol]

        # I < V < X
        if symbol < roman[ind+1]:
            result-= rom[symbol]
        else:
            result+= rom[symbol]

def roman_apply(foundation_year, _foundation_year) -> (str, bool):
    search_res = re.search(roman_nums, foundation_year)
    if search_res is None:
        return foundation_year, _foundation_year

    year = roman_to_digits(search_res.group(1))*100 -1
    # до н. э.
    if 'до' in foundation_year:
        year*= -1

    return str(year), not _foundation_year

In [14]:
data[['foundation_year', '_foundation_year']] = data[['foundation_year', '_foundation_year']].apply(lambda x: roman_apply(*x), axis=1, result_type='expand')

In [15]:
data[data._foundation_year == True]

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,capital_marker,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year,_foundation_year
1,"Респ Башкортостан, г Давлеканово",453400,Россия,Приволжский,Респ,Башкортостан,р-н,Давлекановский,город,Давлеканово,...,1,80419000000,80622101,273,UTC+5,54.2226707,55.0312373,24040,1730-е,True
3,"Респ Башкортостан, г Янаул",452800,Россия,Приволжский,Респ,Башкортостан,р-н,Янаульский,город,Янаул,...,1,80460000000,80659101,264,UTC+5,56.2650146,54.929907,26988,1730-е,True
4,"Брянская обл, г Сельцо",241550,Россия,Центральный,обл,Брянская,,,город,Сельцо,...,0,15425000000,15725000001,3245,UTC+3,53.3739884,34.1059172,17933,1870-е,True
8,"Владимирская обл, Собинский р-н, г Лакинск",601240,Россия,Центральный,обл,Владимирская,р-н,Собинский,город,Лакинск,...,0,17250503000,17650110001,3327,UTC+3,56.0180587,39.956551,15707,15 век,True
11,"Волгоградская обл, г Котово",403801,Россия,Южный,обл,Волгоградская,р-н,Котовский,город,Котово,...,1,18226501000,18626101001,3453,UTC+4,50.3205766,44.8030699,24104,1710-е,True
12,"Вологодская обл, г Бабаево",162480,Россия,Северо-Западный,обл,Вологодская,р-н,Бабаевский,город,Бабаево,...,1,19205501000,19605101,3532,UTC+3,59.3892583,35.9377058,12074,15 век,True
16,"Ивановская обл, г Заволжск",155410,Россия,Центральный,обл,Ивановская,р-н,Заволжский,город,Заволжск,...,1,24205501000,24605101001,3703,UTC+3,57.4918141,42.1375625,12045,19 век,True
19,"Иркутская обл, г Свирск",665420,Россия,Сибирский,обл,Иркутская,,,город,Свирск,...,0,25427000000,25746000001,3851,UTC+8,53.0842668,103.3364093,13649,19 век,True
20,"Респ Кабардино-Балкарская, г Майский",361110,Россия,Северо-Кавказский,Респ,Кабардино-Балкарская,р-н,Майский,город,Майский,...,1,83220501000,83620101,716,UTC+3,43.6281516,44.0517314,26755,19 век,True
28,"Респ Карелия, г Питкяранта",186810,Россия,Северо-Западный,Респ,Карелия,р-н,Питкярантский,город,Питкяранта,...,1,86233501000,86633101001,1035,UTC+3,61.5757191,31.4640557,11484,19 век,True


In [16]:
def cut_endswith_e(year, flag) -> (str, bool):
    if year.endswith('-е'):
        return year[:-2], not flag
    else:
        return year, flag

In [17]:
data[['foundation_year', '_foundation_year']] = data[['foundation_year', '_foundation_year']].apply(lambda x: cut_endswith_e(*x), axis=1, result_type='expand')

In [18]:
data[data._foundation_year == True]

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,capital_marker,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year,_foundation_year
8,"Владимирская обл, Собинский р-н, г Лакинск",601240,Россия,Центральный,обл,Владимирская,р-н,Собинский,город,Лакинск,...,0,17250503000,17650110001,3327,UTC+3,56.0180587,39.956551,15707,15 век,True
12,"Вологодская обл, г Бабаево",162480,Россия,Северо-Западный,обл,Вологодская,р-н,Бабаевский,город,Бабаево,...,1,19205501000,19605101,3532,UTC+3,59.3892583,35.9377058,12074,15 век,True
16,"Ивановская обл, г Заволжск",155410,Россия,Центральный,обл,Ивановская,р-н,Заволжский,город,Заволжск,...,1,24205501000,24605101001,3703,UTC+3,57.4918141,42.1375625,12045,19 век,True
19,"Иркутская обл, г Свирск",665420,Россия,Сибирский,обл,Иркутская,,,город,Свирск,...,0,25427000000,25746000001,3851,UTC+8,53.0842668,103.3364093,13649,19 век,True
20,"Респ Кабардино-Балкарская, г Майский",361110,Россия,Северо-Кавказский,Респ,Кабардино-Балкарская,р-н,Майский,город,Майский,...,1,83220501000,83620101,716,UTC+3,43.6281516,44.0517314,26755,19 век,True
28,"Респ Карелия, г Питкяранта",186810,Россия,Северо-Западный,Респ,Карелия,р-н,Питкярантский,город,Питкяранта,...,1,86233501000,86633101001,1035,UTC+3,61.5757191,31.4640557,11484,19 век,True
30,"Кировская обл, г Малмыж",612920,Россия,Приволжский,обл,Кировская,р-н,Малмыжский,город,Малмыж,...,1,33223501000,33623101001,4307,UTC+3,56.5243844,50.6782739,8265,15 век,True
34,"Респ Крым, г Алушта",298500,Россия,Южный,Респ,Крым,,,город,Алушта,...,0,35403000000,35703000,9100,UTC+3,44.6764304,34.4100624,29078,6 век,True
35,"Респ Крым, Белогорский р-н, г Белогорск",297600,Россия,Южный,Респ,Крым,р-н,Белогорский,город,Белогорск,...,0,35207501000,35607101001,9109,UTC+3,45.057202,34.5999029,16354,13 век,True
36,"Респ Крым, г Евпатория",297402,Россия,Южный,Респ,Крым,,,город,Евпатория,...,0,35409000000,35712000001,9110,UTC+3,45.190629,33.367634,105719,497 год до н. э.,True


In [19]:
def cut_endswith_sent(year, flag) -> (str, bool):
    if year.endswith('век'):
        return str(int(year[:-4])*100 - 1), not flag
    else:
        return year, flag

In [20]:
data[['foundation_year', '_foundation_year']] = data[['foundation_year', '_foundation_year']].apply(lambda x: cut_endswith_sent(*x), axis=1, result_type='expand')

In [21]:
data[data._foundation_year == True].foundation_year

36       497 год до н. э.
37     7 — 6 век до н. э.
61              1677-1678
68               8—9 века
90              9-10 века
100         1378 или 1384
Name: foundation_year, dtype: object

In [22]:
data.at[36, 'foundation_year'] = '-497'
data.at[37, 'foundation_year'] = '500'
data.at[61, 'foundation_year'] = '1678'
data.at[68, 'foundation_year'] = '800'
data.at[90, 'foundation_year'] = '900'
data.at[100, 'foundation_year'] = '1384'
data.drop(columns='_foundation_year', inplace=True)

## Save and upload

In [23]:
data.to_csv('handled_1.csv', encoding='utf-8', index=False)

In [24]:
headers = {'Content-Type': 'text/csv'}
response = requests.post(endpoint,
                         data=open('handled_1.csv' ,'rb'),
                         headers=headers)
response.status_code

201